In [1]:
import time
import torch


In [2]:
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.ToTensor()])

train_data = datasets.FashionMNIST(root='data', train=True, download=True , transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)


100%|██████████| 26.4M/26.4M [00:02<00:00, 9.43MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 202kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.33MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.70MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
import torch
model_with_layers = MLP()
model_with_layers.load_state_dict(torch.load('mlp_model_weights.pth'))
state_dict = torch.load('mlp_model_weights.pth')

<ipython-input-4-fcb69fb45a44>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_with_layers.load_state_dict(torch.load('mlp_model_weights.pth'))
<ipython-input-4-fcb69

In [5]:
def relu(x):
    return torch.max(torch.zeros_like(x), x)

In [13]:


state_dict['fc1.weight'] = state_dict['fc1.weight'].to('cuda')
state_dict['fc1.bias'] = state_dict['fc1.bias'].to('cuda')
state_dict['fc2.weight'] = state_dict['fc2.weight'].to('cuda')
state_dict['fc2.bias'] = state_dict['fc2.bias'].to('cuda')
def Mymodel(x):
    x = x.view(-1, 28 * 28)
    x = relu(x @ state_dict['fc1.weight'].T + state_dict['fc1.bias'])
    x = x @ state_dict['fc2.weight'].T + state_dict['fc2.bias']

    return x

In [ ]:
model = MLP()
model.to("cpu")

start_time = time.time()
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to("cpu")
        outputs = model(images)
end_time = time.time()

cpu_time = end_time - start_time
print(f"Torch on CPU: {cpu_time:.4f} ")


Torch on CPU: 1.9224 


In [ ]:
if torch.cuda.is_available():
    model.to("cuda")
    start_time = time.time()
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to("cuda")
            outputs = model(images)
    end_time = time.time()

    gpu_time = end_time - start_time
    print(f"Torch on GPU: {gpu_time:.4f} ")


Torch on GPU: 1.4352 


In [ ]:

device = torch.device("cpu")

start_time = time.time()
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = Mymodel(images)
end_time = time.time()

cpu_time_custom = end_time - start_time
print(f"Mymodel on CPU: {cpu_time_custom:.4f} ")


Mymodel on CPU: 1.7467 


In [15]:
if torch.cuda.is_available():
    device = torch.device("cuda")

    start_time = time.time()
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to("cuda")

            outputs = Mymodel(images)
    end_time = time.time()

    gpu_time_custom = end_time - start_time
    print(f"Mymodel on GPU: {gpu_time_custom:.4f} ")


Mymodel on GPU: 1.3453 
